# I fucking hate `pandas` and `sklearn` for their messy and inconsistent interfaces
# for their opposition to any extensions of their libraries
# and all that crap.
# I can't even sleep because of how much I hate them.
# I was trying to write things according to their API
# implementing BaseEstimator and TransformerMixin
# and using pipes
# but that makes things even worser
# because of uglyness of this environment!

# I wanted to try pretty obvious tricks and models
# but this environment makes it impossible

In [1]:
import pandas as pd
import numpy as np
import dateutil.parser
import datetime
import sklearn.preprocessing
import collections
from sklearn.feature_extraction.text import TfidfVectorizer


%matplotlib inline

In [2]:
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
X_test = pd.read_csv('X_test.csv')

In [3]:
X_train.dtypes

id                         int64
amount_tsh               float64
date_recorded             object
funder                    object
gps_height                 int64
installer                 object
longitude                float64
latitude                 float64
wpt_name                  object
num_private                int64
basin                     object
subvillage                object
region                    object
region_code                int64
district_code              int64
lga                       object
ward                      object
population                 int64
public_meeting            object
recorded_by               object
scheme_management         object
scheme_name               object
permit                    object
construction_year          int64
extraction_type           object
extraction_type_group     object
extraction_type_class     object
management                object
management_group          object
payment                   object
payment_ty

In [4]:
def date_to_ymd(x):
    res = (np.nan, np.nan, np.nan)
    if isinstance(x, str):
        try:
            x = dateutil.parser.parse(x)
        except:
            pass
    if isinstance(x, datetime.date):
            res = (x.year, x.month, x.day)
    return res

In [5]:
X_train.drop(['id', 'recorded_by', 'num_private'], axis=1, inplace=True)
X_test.drop(['id', 'recorded_by', 'num_private'], axis=1, inplace=True)

In [6]:
(X_train.loc[:,'date_recorded_year'],
 X_train.loc[:,'date_recorded_month'],
 X_train.loc[:,'date_recorded_day']) = zip(*X_train.loc[:,'date_recorded'].map(date_to_ymd))
(X_test.loc[:,'date_recorded_year'],
 X_test.loc[:,'date_recorded_month'],
 X_test.loc[:,'date_recorded_day']) = zip(*X_test.loc[:,'date_recorded'].map(date_to_ymd))

X_train.drop('date_recorded', axis=1, inplace=True)
X_test.drop('date_recorded', axis=1, inplace=True)

In [7]:
X_train.loc[:,'age'] = X_train.loc[:,'date_recorded_year'] - X_train.loc[:,'construction_year']
X_test.loc[:,'age'] = X_test.loc[:,'date_recorded_year'] - X_test.loc[:,'construction_year']

In [8]:
X_train.loc[:,'amount_per_man'] = (X_train.loc[:,'amount_tsh']/X_train.loc[:,'population']).fillna(0)
X_test.loc[:,'amount_per_man'] = (X_test.loc[:,'amount_tsh']/X_test.loc[:,'population']).fillna(0)

In [9]:
X_train.loc[X_train.loc[:,'permit'] == True, 'permit'] = 1
X_train.loc[X_train.loc[:,'permit'] == False, 'permit'] = 0
X_train.loc[:,'permit'].fillna(0.5, inplace=True)
X_test.loc[X_test.loc[:,'permit'] == True, 'permit'] = 1
X_test.loc[X_test.loc[:,'permit'] == False, 'permit'] = 0
X_test.loc[:,'permit'].fillna(0.5, inplace=True)

In [10]:
X_train.loc[X_train.loc[:,'public_meeting'] == True, 'public_meeting'] = 1
X_train.loc[X_train.loc[:,'public_meeting'] == False, 'public_meeting'] = 0
X_train.loc[:,'public_meeting'].fillna(0.5, inplace=True)
X_test.loc[X_test.loc[:,'public_meeting'] == True, 'public_meeting'] = 1
X_test.loc[X_test.loc[:,'public_meeting'] == False, 'public_meeting'] = 0
X_test.loc[:,'public_meeting'].fillna(0.5, inplace=True)

In [11]:
X_train.loc[:,'wpt_name'] = X_train.loc[:,'wpt_name'].notnull() * 1
X_test.loc[:,'wpt_name'] = X_test.loc[:,'wpt_name'].notnull() * 1

In [12]:
X_train.loc[:,'source'] = (
    X_train.loc[:,'source'] + ' '
    + X_train.loc[:,'source_type'] + ' '
    + X_train.loc[:,'source_class']
)
X_test.loc[:,'source'] = (
    X_test.loc[:,'source'] + ' '
    + X_test.loc[:,'source_type'] + ' '
    + X_test.loc[:,'source_class']
)

In [13]:
X_train.loc[:,'quality'] = (
    X_train.loc[:,'water_quality'] + ' '
    + X_train.loc[:,'quality_group']
)
X_test.loc[:,'quality'] = (
    X_test.loc[:,'water_quality'] + ' '
    + X_test.loc[:,'quality_group']
)

In [14]:
X_train.loc[:,'waterpoint_type'] = (
    X_train.loc[:,'waterpoint_type'] + ' '
    + X_train.loc[:,'waterpoint_type_group']
)
X_test.loc[:,'waterpoint_type'] = (
    X_test.loc[:,'waterpoint_type'] + ' '
    + X_test.loc[:,'waterpoint_type_group']
)

In [15]:
X_train.loc[:,'quantity'] = (
    X_train.loc[:,'quantity'] + ' '
    + X_train.loc[:,'quantity_group']
)
X_test.loc[:,'quantity'] = (
    X_test.loc[:,'quantity'] + ' '
    + X_test.loc[:,'quantity_group']
)

In [16]:
X_train.loc[:,'management'] = (
    X_train.loc[:,'management'] + ' '
    + X_train.loc[:,'management_group']
)
X_test.loc[:,'management'] = (
    X_test.loc[:,'management'] + ' '
    + X_test.loc[:,'management_group']
)

In [17]:
X_train.loc[:,'payment'] = (
    X_train.loc[:,'payment'] + ' '
    + X_train.loc[:,'payment_type']
)
X_test.loc[:,'payment'] = (
    X_test.loc[:,'payment'] + ' '
    + X_test.loc[:,'payment_type']
)

In [18]:
X_train.loc[:,'extraction_type'] = (
    X_train.loc[:,'extraction_type'] + ' '
    + X_train.loc[:,'extraction_type_group'] + ' '
    + X_train.loc[:,'extraction_type_class']
)
X_test.loc[:,'extraction_type'] = (
    X_test.loc[:,'extraction_type'] + ' '
    + X_test.loc[:,'extraction_type_group'] + ' '
    + X_test.loc[:,'extraction_type_class']
)

In [19]:
drop_cats = ['source_type', 'source_class',
              'water_quality', 'quality_group',
              'quantity_group', 'waterpoint_type_group',
              'payment_type', 'extraction_type_group',
              'extraction_type_class', 'management_group'
              ]
X_train.drop(drop_cats, axis=1, inplace=True, errors='ignore')
X_test.drop(drop_cats, axis=1, inplace=True, errors='ignore')

In [20]:
X_train.iloc[0,:]

amount_tsh                                              6000
funder                                                 Roman
gps_height                                              1390
installer                                              Roman
longitude                                            34.9381
latitude                                            -9.85632
wpt_name                                                   1
basin                                             Lake Nyasa
subvillage                                          Mnyusi B
region                                                Iringa
region_code                                               11
district_code                                              5
lga                                                   Ludewa
ward                                                Mundindi
population                                               109
public_meeting                                             1
scheme_management       

In [21]:
text_features = ['funder', 'installer', 'basin',
                'subvillage', 'region',
                'lga', 'ward', 
                 'quality',
                 #'water_quality', 'quality_group',
                'scheme_management', 'scheme_name',
                'extraction_type', #'extraction_type_group',
                'management',# 'management_group', 
                 'payment',# 'payment_type',
                'quantity',# 'quantity_group',
                'source',# 'source_type', 'source_class',
                 'waterpoint_type', #'waterpoint_type_group',
                #'date_recorded_year', 'construction_year',
               ]
text_features_vectorizers = dict()
text_feats_train = []
text_feats_test = []

In [22]:
for cat in text_features:
    if X_train.loc[:,cat].dtype == object:
        X_train.loc[:,cat].fillna('unknown', inplace=True)
        X_train.loc[X_train.loc[:,cat].str.contains('other') == True, cat] = 'unknown'
        X_test.loc[:,cat].fillna('unknown', inplace=True)
        X_test.loc[X_test.loc[:,cat].str.contains('other') == True, cat] = 'unknown'
    else:
        X_train.loc[:,cat].fillna(0)
        X_test.loc[:,cat].fillna(0)
    xtrainvals = (X_train[:,cat].unique())
    #labels = X_test.loc[:,cat].unique()
    #labelscnt = np.zeros((3, labels.size))
    #def label_to_i(lbl):
    #    if  pd.isnull(lbl) or lbl == 'unknown' or lbl == 'other' or lbl == 'others' or lbl == 0:
    #        return 0
    #    i = labels.searchsorted(lbl)
    #    if i < labels.size and labels[i] == lbl:
    #        return 1 + i
    #    return 0
    #X_train.loc[:,cat] = X_train.loc[:,cat].map(label_to_i)
    #X_test.loc[:,cat] = X_test.loc[:,cat].map(label_to_i)
    v = TfidfVectorizer(max_features=20)
    v.fit(X_test.loc[:,cat])
    #X_train.loc[:,cat] = v.transform(X_train.loc[:,cat])
    #X_test.loc[:,cat] = v.transform(X_test.loc[:,cat])
    tr = pd.DataFrame(v.transform(X_train.loc[:,cat]).toarray()).add_prefix(cat + '_')
    te = pd.DataFrame(v.transform(X_test.loc[:,cat]).toarray()).add_prefix(cat + '_')
    text_feats_train.append(tr)
    text_feats_test.append(te)



X_train = pd.concat([X_train.drop(text_features, axis=1, errors='ignore')] + text_feats_train,
                    axis=1)
X_test = pd.concat([X_test.drop(text_features, axis=1, errors='ignore')] + text_feats_test,
                    axis=1)


In [23]:
X_train.head()

,amount_tsh,gps_height,longitude,latitude,wpt_name,region_code,district_code,population,public_meeting,permit,...,waterpoint_type_1,waterpoint_type_2,waterpoint_type_3,waterpoint_type_4,waterpoint_type_5,waterpoint_type_6,waterpoint_type_7,waterpoint_type_8,waterpoint_type_9,waterpoint_type_10
0,6000.0,1390,34.938093,-9.856322,1,11,5,109,1.0,0.0,...,0.707107,0.0,0.0,0.0,0.000000,0.0,0.0,0.707107,0.0,0.0
1,0.0,1399,34.698766,-2.147466,1,20,2,280,0.5,1.0,...,0.707107,0.0,0.0,0.0,0.000000,0.0,0.0,0.707107,0.0,0.0
2,25.0,686,37.460664,-3.821329,1,21,4,250,1.0,1.0,...,0.565057,0.0,0.0,0.0,0.601183,0.0,0.0,0.565057,0.0,0.0
3,0.0,263,38.486161,-11.155298,1,90,63,58,1.0,1.0,...,0.565057,0.0,0.0,0.0,0.601183,0.0,0.0,0.565057,0.0,0.0
4,0.0,0,31.130847,-1.825359,1,18,1,0,1.0,1.0,...,0.707107,0.0,0.0,0.0,0.000000,0.0,0.0,0.707107,0.0,0.0


In [24]:
[(c, X_train[c].dtype) for c in X_train.columns if not np.issubdtype(X_train[c].dtype, np.number)]

[]

In [25]:
def split_train_cv_test(X, y, proportions=(.75, .25/2, .25/2)):
    # expecting $y$ to be numpy array
    outs = np.unique(y)
    proportions = np.array(proportions)
    classes = [np.where(y == i)[0] for i in outs]
    xparts = [[] for _ in proportions]
    yparts = [[] for _ in proportions]
    for cidx in classes:
        cidx = sklearn.utils.shuffle(cidx)
        cprops = cidx.size * proportions
        cprops = cprops.astype(int)
        cprops[-1] = cidx.size - cprops[:-1].sum()
        # print(cidx)
        cx = X.iloc[cidx,:]
        cy = y[cidx]
        for xpart, ypart, sz in zip(xparts, yparts, cprops):
            xpart.append(cx.iloc[:sz,:])
            ypart.append(cy[:sz])
            cx, cy = cx.iloc[sz:,:], cy[sz:]
    xparts = [pd.concat(xpart) for xpart in xparts]
    yparts = [np.concatenate(ypart) for ypart in yparts]
    parts = xparts + yparts
    return parts


class OutcomeFrequences:
    def __init__(self, groupby, drop=False, return_only_fq=False):
        self.groupby = groupby
        self.drop = drop
        self.cnts_ = None
        self.fq_cols_ = None
        self.return_only_fq = return_only_fq
        self.unknowns_ = 0
        
    def fit(self, X, y=None):
        possible_outcomes = [_.split()[-1] if isinstance(_, str) else _ for _ in np.unique(y)]
        n_outs = len(possible_outcomes)
        self.cnts_ = collections.defaultdict(lambda: np.zeros(n_outs))
        # print(self.groupby)
        igroupby = np.array([X.columns.get_loc(_) for _ in self.groupby])
        for i in range(X.shape[0]):
            # assuming $y$ is label-encoded
            keys, out = tuple(X.iloc[i, igroupby].values), y[i]
            self.cnts_[keys][out] += 1 # no of `(keys, out)` occurences
        for k in self.cnts_:
            self.cnts_[k] //= self.cnts_[k].sum()
            
        self.possible_outcomes_ = possible_outcomes
        self.fq_cols_ = ['_'.join(self.groupby + [str(out), 'fq'],) for out in possible_outcomes]
        return self
    
    def transform(self, X):
        self.unknowns_ = 0
        igroupby = np.array([X.columns.get_loc(_) for _ in self.groupby])
        new_cols = [tuple([None for out in self.possible_outcomes_]) for _ in range(X.shape[0])]
        for i in range(X.shape[0]):
            keys = tuple(X.iloc[i, igroupby].values)
            if keys in self.cnts_:
                new_cols[i] = tuple([self.cnts_[keys][out] for out in self.possible_outcomes_])
            else:
                self.unknowns_ += 1
        self.unknowns_ /= X.size
        # print(str(X)[:100])
        # print(str(new_cols)[:100])
        # print(str(self.fq_cols_[:100]))
        new_cols = pd.DataFrame(new_cols, columns=self.fq_cols_)
        for c in self.fq_cols_:
            new_cols.loc[:,c].fillna(0, inplace=True) # new_cols.loc[:,c].median(), inplace=True)
        if self.return_only_fq:
            return new_cols
        X = pd.concat((X, new_cols,), axis=1)
        if self.drop:
            X.drop(self.groupby, axis=1, inplace=True, errors='ignore')
        return X

fqpreprs = [(catcol + '_fq', OutcomeFrequences([catcol], drop=False))
    for catcol in ['installer', 'funder', 'subvillage', 'basin', 'extraction',
                   'date_recorded_month', 'region', 'ward', 'source']
]
xtr = X_train # ref
xte = X_test # ref
y_enc1 = sklearn.preprocessing.LabelEncoder().fit(y_train.iloc[:,1])
ytr = y_enc1.transform(y_train.iloc[:,1])
#for i in range(len(fqpreprs)):
#    print('fitting %s' % fqpreprs[i][0])
#    fqpreprs[i][1].fit(xtr, ytr)
#    xtr = fqpreprs[i][1].transform(xtr)
#    print('applying %s to test' % fqpreprs[i][0])
#    xte = fqpreprs[i][1].transform(xte)


xtr, xcv, xva, ytr, ycv, yva = (
    split_train_cv_test(xtr, ytr, proportions=(.7, .15, .15))
)
#y_bin = sklearn.preprocessing.LabelBinarizer().fit(ytr)
#display(
#    np.where(ytr != 0)[0][:40],
#    np.where(ycv != 0)[0][:40],
#    np.where(yva != 0)[0][:40],)
#ytr = y_bin.transform(ytr)
#ycv = y_bin.transform(ycv)
#yva = y_bin.transform(yva)

In [26]:
def y_to_df(y, ids):
    #y = y_bin.inverse_transform(y)
    y = y_enc1.inverse_transform(y)
    df = pd.DataFrame({'status_group': y}, index=ids)
    df.index.name = 'id'
    return df

In [27]:
xtr.columns.tolist()

['amount_tsh',
 'gps_height',
 'longitude',
 'latitude',
 'wpt_name',
 'region_code',
 'district_code',
 'population',
 'public_meeting',
 'permit',
 'construction_year',
 'date_recorded_year',
 'date_recorded_month',
 'date_recorded_day',
 'age',
 'amount_per_man',
 'funder_0',
 'funder_1',
 'funder_2',
 'funder_3',
 'funder_4',
 'funder_5',
 'funder_6',
 'funder_7',
 'funder_8',
 'funder_9',
 'funder_10',
 'funder_11',
 'funder_12',
 'funder_13',
 'funder_14',
 'funder_15',
 'funder_16',
 'funder_17',
 'funder_18',
 'funder_19',
 'installer_0',
 'installer_1',
 'installer_2',
 'installer_3',
 'installer_4',
 'installer_5',
 'installer_6',
 'installer_7',
 'installer_8',
 'installer_9',
 'installer_10',
 'installer_11',
 'installer_12',
 'installer_13',
 'installer_14',
 'installer_15',
 'installer_16',
 'installer_17',
 'installer_18',
 'installer_19',
 'basin_0',
 'basin_1',
 'basin_2',
 'basin_3',
 'basin_4',
 'basin_5',
 'basin_6',
 'basin_7',
 'basin_8',
 'basin_9',
 'basin_10',


In [28]:
X_train.to_csv('X_train_prepr2.csv')
X_test.to_csv('X_test_prepr2.csv')
# pd.DataFrame(ytr, columns=y_enc1.classes_).to_csv('y_train_prepr2.csv')
pd.Series(ytr).to_csv('y_train_prepr2.csv')
pd.Series(y_enc1.classes_).to_csv('y_classes2.csv')

In [29]:
import catboost

In [30]:
clf = catboost.CatBoostClassifier(1800, loss_function='MultiClass',
                                  eval_metric='Accuracy',
                                  calc_feature_importances=True)

In [32]:
clf.fit(xtr, ytr, cat_features=[xtr.columns.get_loc(c) for c in text_features if c in xtr.columns],
        eval_set=(xcv, ycv), verbose=True)

Borders generated
0:	learn 0.6950550772	test 0.6956668163	bestTest 0.6956668163	passed: 0.322 sec	total: 817ms	remaining: 24m 30s
1:	learn 0.7048198567	test 0.7056578357	bestTest 0.7056578357	passed: 0.266 sec	total: 1.08s	remaining: 16m 13s
2:	learn 0.7054932897	test 0.7083520431	bestTest 0.7083520431	passed: 0.266 sec	total: 1.35s	remaining: 13m 28s
3:	learn 0.7021261244	test 0.7041984733	bestTest 0.7083520431	passed: 0.264 sec	total: 1.61s	remaining: 12m 4s
4:	learn 0.6988311126	test 0.700830714	bestTest 0.7083520431	passed: 0.271 sec	total: 1.89s	remaining: 11m 16s
5:	learn 0.7037135023	test 0.7056578357	bestTest 0.7083520431	passed: 0.267 sec	total: 2.15s	remaining: 10m 43s
6:	learn 0.7037856559	test 0.7056578357	bestTest 0.7083520431	passed: 0.309 sec	total: 2.46s	remaining: 10m 30s
7:	learn 0.702342585	test 0.7034126628	bestTest 0.7083520431	passed: 0.278 sec	total: 2.74s	remaining: 10m 13s
8:	learn 0.702871711	test 0.7048720251	bestTest 0.7083520431	passed: 0.259 sec	total: 3s	

74:	learn 0.7236278801	test 0.7237314773	bestTest 0.7237314773	passed: 0.296 sec	total: 20.8s	remaining: 7m 59s
75:	learn 0.7239645967	test 0.723843736	bestTest 0.723843736	passed: 0.246 sec	total: 21.1s	remaining: 7m 58s
76:	learn 0.7238443408	test 0.7240682533	bestTest 0.7240682533	passed: 0.24 sec	total: 21.3s	remaining: 7m 57s
77:	learn 0.7239886478	test 0.7242927705	bestTest 0.7242927705	passed: 0.347 sec	total: 21.7s	remaining: 7m 58s
78:	learn 0.7240608014	test 0.7246295465	bestTest 0.7246295465	passed: 0.282 sec	total: 22s	remaining: 7m 58s
79:	learn 0.7240848526	test 0.7248540638	bestTest 0.7248540638	passed: 0.32 sec	total: 22.3s	remaining: 7m 59s
80:	learn 0.7241570061	test 0.7249663224	bestTest 0.7249663224	passed: 0.266 sec	total: 22.5s	remaining: 7m 58s
81:	learn 0.725744384	test 0.7253030983	bestTest 0.7253030983	passed: 0.253 sec	total: 22.8s	remaining: 7m 57s
82:	learn 0.7256481793	test 0.7255276156	bestTest 0.7255276156	passed: 0.312 sec	total: 23.1s	remaining: 7m 58s

148:	learn 0.7429650296	test 0.7400089807	bestTest 0.7403457566	passed: 0.287 sec	total: 41.3s	remaining: 7m 37s
149:	learn 0.7432776949	test 0.7404580153	bestTest 0.7404580153	passed: 0.279 sec	total: 41.5s	remaining: 7m 37s
150:	learn 0.7433979508	test 0.7405702739	bestTest 0.7405702739	passed: 0.3 sec	total: 41.9s	remaining: 7m 37s
151:	learn 0.7435903603	test 0.7407947912	bestTest 0.7407947912	passed: 0.281 sec	total: 42.1s	remaining: 7m 36s
152:	learn 0.7437346674	test 0.7407947912	bestTest 0.7407947912	passed: 0.313 sec	total: 42.4s	remaining: 7m 36s
153:	learn 0.7437106162	test 0.7409070498	bestTest 0.7409070498	passed: 0.309 sec	total: 42.8s	remaining: 7m 36s
154:	learn 0.7440954351	test 0.7410193085	bestTest 0.7410193085	passed: 0.286 sec	total: 43s	remaining: 7m 36s
155:	learn 0.7445283563	test 0.7412438258	bestTest 0.7412438258	passed: 0.29 sec	total: 43.3s	remaining: 7m 36s
156:	learn 0.7446245611	test 0.7413560844	bestTest 0.7413560844	passed: 0.287 sec	total: 43.6s	remain

221:	learn 0.7523930925	test 0.746856758	bestTest 0.746856758	passed: 0.31 sec	total: 1m 2s	remaining: 7m 25s
222:	learn 0.7528741161	test 0.7470812753	bestTest 0.7470812753	passed: 0.302 sec	total: 1m 2s	remaining: 7m 25s
223:	learn 0.7531146279	test 0.7475303098	bestTest 0.7475303098	passed: 0.307 sec	total: 1m 3s	remaining: 7m 25s
224:	learn 0.7531627303	test 0.7474180512	bestTest 0.7475303098	passed: 0.333 sec	total: 1m 3s	remaining: 7m 25s
225:	learn 0.7533791909	test 0.7474180512	bestTest 0.7475303098	passed: 0.305 sec	total: 1m 3s	remaining: 7m 25s
226:	learn 0.7534272933	test 0.7477548271	bestTest 0.7477548271	passed: 0.266 sec	total: 1m 4s	remaining: 7m 24s
227:	learn 0.7534753956	test 0.7474180512	bestTest 0.7477548271	passed: 0.32 sec	total: 1m 4s	remaining: 7m 24s
228:	learn 0.7537159075	test 0.7475303098	bestTest 0.7477548271	passed: 0.303 sec	total: 1m 4s	remaining: 7m 24s
229:	learn 0.7536918563	test 0.7473057925	bestTest 0.7477548271	passed: 0.296 sec	total: 1m 5s	remai

294:	learn 0.7583337342	test 0.7519083969	bestTest 0.7519083969	passed: 0.278 sec	total: 1m 25s	remaining: 7m 13s
295:	learn 0.7583818365	test 0.7520206556	bestTest 0.7520206556	passed: 0.299 sec	total: 1m 25s	remaining: 7m 13s
296:	learn 0.7586223484	test 0.7522451729	bestTest 0.7522451729	passed: 0.276 sec	total: 1m 25s	remaining: 7m 13s
297:	learn 0.7587666554	test 0.7521329142	bestTest 0.7522451729	passed: 0.259 sec	total: 1m 25s	remaining: 7m 12s
298:	learn 0.7587426043	test 0.7523574315	bestTest 0.7523574315	passed: 0.275 sec	total: 1m 26s	remaining: 7m 12s
299:	learn 0.7586704507	test 0.7523574315	bestTest 0.7523574315	passed: 0.274 sec	total: 1m 26s	remaining: 7m 12s
300:	learn 0.7586945019	test 0.7526942075	bestTest 0.7526942075	passed: 0.29 sec	total: 1m 26s	remaining: 7m 11s
301:	learn 0.758838809	test 0.7526942075	bestTest 0.7526942075	passed: 0.237 sec	total: 1m 26s	remaining: 7m 11s
302:	learn 0.758838809	test 0.7525819488	bestTest 0.7526942075	passed: 0.252 sec	total: 1m

...

1456:	learn 0.8014093992	test 0.782779524	bestTest 0.782779524	passed: 0.291 sec	total: 6m 54s	remaining: 1m 37s
1457:	learn 0.8014334504	test 0.782779524	bestTest 0.782779524	passed: 0.253 sec	total: 6m 54s	remaining: 1m 37s
1458:	learn 0.8014093992	test 0.7828917827	bestTest 0.7828917827	passed: 0.273 sec	total: 6m 55s	remaining: 1m 37s
1459:	learn 0.8014093992	test 0.782779524	bestTest 0.7828917827	passed: 0.302 sec	total: 6m 55s	remaining: 1m 36s
1460:	learn 0.8014815527	test 0.782779524	bestTest 0.7828917827	passed: 0.289 sec	total: 6m 55s	remaining: 1m 36s
1461:	learn 0.8015296551	test 0.7832285586	bestTest 0.7832285586	passed: 0.265 sec	total: 6m 56s	remaining: 1m 36s
1462:	learn 0.8016258598	test 0.7832285586	bestTest 0.7832285586	passed: 0.291 sec	total: 6m 56s	remaining: 1m 35s
1463:	learn 0.801914474	test 0.7833408172	bestTest 0.7833408172	passed: 0.264 sec	total: 6m 56s	remaining: 1m 35s
1464:	learn 0.8019866275	test 0.7834530759	bestTest 0.7834530759	passed: 0.268 sec	tota

1528:	learn 0.802708163	test 0.7844634037	bestTest 0.7848001796	passed: 0.236 sec	total: 7m 15s	remaining: 1m 17s
1529:	learn 0.8028284189	test 0.784687921	bestTest 0.7848001796	passed: 0.276 sec	total: 7m 15s	remaining: 1m 16s
1530:	learn 0.8027562653	test 0.784687921	bestTest 0.7848001796	passed: 0.275 sec	total: 7m 15s	remaining: 1m 16s
1531:	learn 0.8027562653	test 0.7844634037	bestTest 0.7848001796	passed: 0.287 sec	total: 7m 16s	remaining: 1m 16s
1532:	learn 0.8029246236	test 0.7851369555	bestTest 0.7851369555	passed: 0.288 sec	total: 7m 16s	remaining: 1m 15s
1533:	learn 0.8029005724	test 0.7851369555	bestTest 0.7851369555	passed: 0.318 sec	total: 7m 16s	remaining: 1m 15s
1534:	learn 0.8029005724	test 0.7851369555	bestTest 0.7851369555	passed: 0.264 sec	total: 7m 16s	remaining: 1m 15s
1535:	learn 0.8028765212	test 0.7849124383	bestTest 0.7851369555	passed: 0.271 sec	total: 7m 17s	remaining: 1m 15s
1536:	learn 0.8029005724	test 0.7850246969	bestTest 0.7851369555	passed: 0.271 sec	

1600:	learn 0.8044638992	test 0.7851369555	bestTest 0.7858105074	passed: 0.273 sec	total: 7m 35s	remaining: 56.6s
1601:	learn 0.8044638992	test 0.7851369555	bestTest 0.7858105074	passed: 0.341 sec	total: 7m 36s	remaining: 56.4s
1602:	learn 0.8045120015	test 0.7851369555	bestTest 0.7858105074	passed: 0.281 sec	total: 7m 36s	remaining: 56.1s
1603:	learn 0.804439848	test 0.7850246969	bestTest 0.7858105074	passed: 0.312 sec	total: 7m 36s	remaining: 55.8s
1604:	learn 0.8044157968	test 0.7850246969	bestTest 0.7858105074	passed: 0.258 sec	total: 7m 36s	remaining: 55.5s
1605:	learn 0.8043917456	test 0.7850246969	bestTest 0.7858105074	passed: 0.298 sec	total: 7m 37s	remaining: 55.2s
1606:	learn 0.8045120015	test 0.7850246969	bestTest 0.7858105074	passed: 0.238 sec	total: 7m 37s	remaining: 54.9s
1607:	learn 0.8045120015	test 0.7851369555	bestTest 0.7858105074	passed: 0.279 sec	total: 7m 37s	remaining: 54.6s
1608:	learn 0.8043917456	test 0.7852492142	bestTest 0.7858105074	passed: 0.266 sec	total:

1673:	learn 0.8060272259	test 0.7861472833	bestTest 0.7863718006	passed: 0.287 sec	total: 7m 56s	remaining: 35.9s
1674:	learn 0.8060031748	test 0.786259542	bestTest 0.7863718006	passed: 0.319 sec	total: 7m 56s	remaining: 35.6s
1675:	learn 0.8060031748	test 0.7861472833	bestTest 0.7863718006	passed: 0.248 sec	total: 7m 57s	remaining: 35.3s
1676:	learn 0.8059791236	test 0.7861472833	bestTest 0.7863718006	passed: 0.318 sec	total: 7m 57s	remaining: 35s
1677:	learn 0.8059791236	test 0.786259542	bestTest 0.7863718006	passed: 0.305 sec	total: 7m 57s	remaining: 34.7s
1678:	learn 0.80590697	test 0.7863718006	bestTest 0.7863718006	passed: 0.23 sec	total: 7m 57s	remaining: 34.4s
1679:	learn 0.8058588677	test 0.7863718006	bestTest 0.7863718006	passed: 0.282 sec	total: 7m 58s	remaining: 34.2s
1680:	learn 0.8058348165	test 0.786259542	bestTest 0.7863718006	passed: 0.299 sec	total: 7m 58s	remaining: 33.9s
1681:	learn 0.8058588677	test 0.786259542	bestTest 0.7863718006	passed: 0.27 sec	total: 7m 58s	r

1746:	learn 0.806700659	test 0.7865963179	bestTest 0.7870453525	passed: 0.29 sec	total: 8m 17s	remaining: 15.1s
1747:	learn 0.8069171196	test 0.7870453525	bestTest 0.7870453525	passed: 0.244 sec	total: 8m 17s	remaining: 14.8s
1748:	learn 0.8069411708	test 0.7871576111	bestTest 0.7871576111	passed: 0.231 sec	total: 8m 17s	remaining: 14.5s
1749:	learn 0.806965222	test 0.7872698698	bestTest 0.7872698698	passed: 0.288 sec	total: 8m 17s	remaining: 14.2s
1750:	learn 0.8069892732	test 0.7872698698	bestTest 0.7872698698	passed: 0.281 sec	total: 8m 18s	remaining: 13.9s
1751:	learn 0.8070854779	test 0.7872698698	bestTest 0.7872698698	passed: 0.268 sec	total: 8m 18s	remaining: 13.7s
1752:	learn 0.8071576314	test 0.7872698698	bestTest 0.7872698698	passed: 0.261 sec	total: 8m 18s	remaining: 13.4s
1753:	learn 0.8071576314	test 0.7873821284	bestTest 0.7873821284	passed: 0.248 sec	total: 8m 18s	remaining: 13.1s
1754:	learn 0.8071335803	test 0.7873821284	bestTest 0.7873821284	passed: 0.285 sec	total: 8

In [33]:
clf.score(xcv, ycv)

0.78816793893129766

In [34]:
clf.score(xva, yva)

0.79100291676015255